In [1]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    DataConfig,
    LoadConfig,
    get_load_config_from_yaml,
    combine_train_datasets
)
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import yaml
import core.assignment_models as Models
import core
import keras
import tensorflow as tf


PLOTS_DIR = f"plots/mixed_model_HLF/"
MODEL_DIR = f"models/mixed_model_HLF"
#MODEL_DIR = f"../models/FeatureConcatTransformer_HLF_d256_l10_h8"
CONFIG_PATH = "../config/workspace_config_global_features.yaml"

import os

if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR, exist_ok=True)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR, exist_ok=True)

load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessorNominal = DataPreprocessor(load_config)
DataProcessorToponium = DataPreprocessor(load_config)

with open(CONFIG_PATH, "r") as file:
    data_configs = yaml.safe_load(file)

plt.rcParams.update({"font.size": 14})

data_config = DataProcessorNominal.load_from_npz(data_configs["data_path"]["nominal"], max_events=2000000)
DataProcessorToponium.load_from_npz(data_configs["data_path"]["toponium"])

X_train, y_train  = DataProcessorNominal.get_data()
X_nominal, y_nominal = DataProcessorNominal.get_data()
X_toponium, y_toponium = DataProcessorToponium.get_data()
X_train, y_train = combine_train_datasets(
    [X_nominal, X_toponium],
    [y_nominal, y_toponium],
    weights=[0.5, 0.5],
)

2025-12-02 15:43:10.356693: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764686590.378585 1123144 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764686590.385696 1123144 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764686590.403110 1123144 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764686590.403126 1123144 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764686590.403129 1123144 computation_placer.cc:177] computation placer alr

In [2]:
TransformerMatcher = Models.FeatureConcatTransformer(data_config, name=r"Transformer")

TransformerMatcher.build_model(
    hidden_dim=128,
    num_layers=8,
    num_heads=8,
    dropout_rate=0.1,
    compute_HLF=True,
)

TransformerMatcher.adapt_normalization_layers(X_train)

TransformerMatcher.compile_model(
    loss={
        "assignment": core.utils.AssignmentLoss(lambda_excl=0.0),
    },
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-4),
    metrics={
        "assignment": [core.utils.AssignmentAccuracy()],
    },
)
TransformerMatcher.model.summary()

FeatureConcatTransformer is designed for classification tasks; regression targets will be ignored.


I0000 00:00:1764686695.555119 1123144 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15511 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:65:00.0, compute capability: 6.0


Building model without regression output.
Submodel inputs:  [<KerasTensor shape=(None, 6, 9), dtype=float32, sparse=False, ragged=False, name=jet_inputs>, <KerasTensor shape=(None, 2, 4), dtype=float32, sparse=False, ragged=False, name=lep_inputs>, <KerasTensor shape=(None, 1, 3), dtype=float32, sparse=False, ragged=False, name=met_inputs>]
Input tensor:  <KerasTensor shape=(None, 6, 9), dtype=float32, sparse=False, ragged=False, name=jet_inputs> jet_inputs
Input tensor:  <KerasTensor shape=(None, 2, 4), dtype=float32, sparse=False, ragged=False, name=lep_inputs> lep_inputs
Input tensor:  <KerasTensor shape=(None, 1, 3), dtype=float32, sparse=False, ragged=False, name=met_inputs> met_inputs


I0000 00:00:1764686699.130983 1125822 service.cc:152] XLA service 0x7efc08003ec0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764686699.131008 1125822 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2025-12-02 15:44:59.154227: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1764686699.205401 1125822 cuda_dnn.cc:529] Loaded cuDNN version 91500


 140/1691 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step

...

Model: "FeatureConcatTransformerModel"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ jet_inputs          │ (None, 6, 9)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_inputs          │ (None, 2, 4)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_input_transform │ (None, 6, 9)      │          0 │ jet_inputs[0][0]  │
│ (InputPtEtaPhiELay… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_mask            │ (None, 6)         │          0 │ jet_inputs[0][0]  │
│ (GenerateMask)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_input_transform │ (None, 2, 4)      │          0 │ lep_inputs[0][0]  │
│ (InputPtEtaPhiELay… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_inputs          │ (None, 1, 3)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ compute_high_level… │ (None, 6, 2, 2)   │          0 │ jet_input_transf… │
│ (ComputeHighLevelF… │                   │            │ jet_mask[0][0],   │
│                     │                   │            │ lep_input_transf… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_input_transform │ (None, 1, 2)      │          0 │ met_inputs[0][0]  │
│ (InputMetPhiLayer)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hlf_input_normaliz… │ (None, 6, 2, 2)   │          9 │ compute_high_lev… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ met_input_normaliz… │ (None, 1, 2)      │          5 │ met_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lep_input_normaliz… │ (None, 2, 4)      │          9 │ lep_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ jet_input_normaliz… │ (None, 6, 9)      │         19 │ jet_input_transf… │
│ (Normalization)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 6, 4)      │          0 │ hlf_input_normal… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 2)         │          0 │ met_input_normal… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 8)         │          0 │ lep_input_normal… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 6, 13)     │          0 │ jet_input_normal… │
│ (Concatenate)       │                   │            │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ repeat_vector       │ (None, 6, 2)      │          0 │ flatten[0][0]     │
│ (RepeatVector)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 274,455 (1.05 MB)

 Trainable params: 274,413 (1.05 MB)

 Non-trainable params: 42 (184.00 B)

In [3]:
TransformerMatcher.train_model(
    epochs=50,
    X_train=X_train,
    y_train=y_train,
    sample_weights=core.utils.compute_sample_weights(X_train, y_train),
    batch_size=1024,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.5,
            patience=5,
            verbose=1,
            mode="min",
            min_lr=1e-6,
        ),
    ],
)

Epoch 1/50


2025-12-02 15:45:38.537618: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert
2025-12-02 15:45:59.096836: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'loop_xor_fusion', 60 bytes spill stores, 60 bytes spill loads



2752/2753 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.1188 - loss: 0.1437

2025-12-02 15:46:53.403399: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert


2753/2753 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - accuracy: 0.1189 - loss: 0.1437

2025-12-02 15:47:14.142126: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert
2025-12-02 15:47:19.178536: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert


2753/2753 ━━━━━━━━━━━━━━━━━━━━ 123s 30ms/step - accuracy: 0.1189 - loss: 0.1437 - val_accuracy: 0.6414 - val_loss: 0.0953 - learning_rate: 1.0000e-04
Epoch 2/50
2753/2753 ━━━━━━━━━━━━━━━━━━━━ 56s 20ms/step - accuracy: 0.6046 - loss: 0.1003 - val_accuracy: 0.6875 - val_loss: 0.0882 - learning_rate: 1.0000e-04
Epoch 3/50
2753/2753 ━━━━━━━━━━━━━━━━━━━━ 56s 20ms/step - accuracy: 0.6520 - loss: 0.0930 - val_accuracy: 0.7017 - val_loss: 0.0852 - learning_rate: 1.0000e-04
Epoch 4/50
2753/2753 ━━━━━━━━━━━━━━━━━━━━ 58s 21ms/step - accuracy: 0.6709 - loss: 0.0901 - val_accuracy: 0.7112 - val_loss: 0.0844 - learning_rate: 1.0000e-04
Epoch 5/50
2753/2753 ━━━━━━━━━━━━━━━━━━━━ 56s 20ms/step - accuracy: 0.6825 - loss: 0.0883 - val_accuracy: 0.7161 - val_loss: 0.0831 - learning_rate: 1.0000e-04
Epoch 6/50
2753/2753 ━━━━━━━━━━━━━━━━━━━━ 57s 21ms/step - accuracy: 0.6884 - loss: 0.0873 - val_accuracy: 0.7196 - val_loss: 0.0824 - learning_rate: 1.0000e-04
Epoch 7/50
2753/2753 ━━━━━━━━━━━━━━━━━━━━ 56s 20ms

In [4]:
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR, exist_ok=True)
TransformerMatcher.save_model(f"{MODEL_DIR}/model.keras")

Model saved to models/mixed_model_HLF/model.keras
